In [666]:
import nbformat

In [667]:
# with open("kjv.ipynb", "r", encoding="utf-8") as f:
#     notebook = nbformat.read(f, as_version=4)

# for cell in notebook.cells:
#     if "outputs" in cell:
#         cell["outputs"] = []
#     if "execution_count" in cell:
#         cell["execution_count"] = None

# with open("kjv_clean.ipynb", "w", encoding="utf-8") as f:
#     nbformat.write(notebook, f)

In [1]:
import json
from tqdm import tqdm
from collections import defaultdict

import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_rows = 50

In [3]:
df = pd.read_csv("../data/kjv_cr.csv", dtype=str)

In [4]:
df = df[~df[['book_name_source_max', 'book_name_target_max']].notna().any(axis=1)].reset_index(drop=True)

df = df[['book_name_source_min', 'chapter_number_source_min', 'verse_number_source_min',
        'book_name_target_min', 'chapter_number_target_min', 'verse_number_target_min']]

df.rename(columns={'book_name_source_min': 'book_name_source', 
                'chapter_number_source_min': 'chapter_number_source', 
                'verse_number_source_min': 'verse_number_source',
                'book_name_target_min': 'book_name_target', 
                'chapter_number_target_min': 'chapter_number_target', 
                'verse_number_target_min': 'verse_number_target'}, 
        inplace=True)

df.dropna(inplace=True)

df[['chapter_number_source', 'verse_number_source', 'chapter_number_target', 'verse_number_target']] = df[
    ['chapter_number_source', 'verse_number_source', 'chapter_number_target', 'verse_number_target']
].astype(int)

df[['book_name_source', 'book_name_target']] = df[
    ['book_name_source', 'book_name_target']
].astype(str)

In [672]:
# df['self_reference'] = (
#     (df['book_name_source'] == df['book_name_target']) &
#     (df['chapter_number_source'] == df['chapter_number_target']) &
#     (df['verse_number_source'] == df['verse_number_target'])
# )

# print(df[df['self_reference']])

In [673]:
# for k, v in df.items():
#     print(k, v.dtype)

In [6]:
print(df)

       book_name_source  chapter_number_source  verse_number_source  \
0               Genesis                      1                    1   
1               Genesis                      1                    1   
2               Genesis                      1                    1   
3               Genesis                      1                    1   
4               Genesis                      1                    1   
...                 ...                    ...                  ...   
256644       Revelation                     22                   21   
256645       Revelation                     22                   21   
256646       Revelation                     22                   21   
256647       Revelation                     22                   21   
256648       Revelation                     22                   21   

       book_name_target  chapter_number_target  verse_number_target  
0                1 John                      1                    1  
1      

In [7]:
cross_ref_dict = defaultdict(lambda: defaultdict(int))

for _, row in df.iterrows():
    source = row["book_name_source"]
    target = row["book_name_target"]
    cross_ref_dict[source][target] += 1
    
dict_cr = {k: dict(v) for k, v in cross_ref_dict.items()}

In [8]:
print(dict_cr)

{'Genesis': {'1 John': 33, 'Psalms': 786, 'Isaiah': 440, 'Ephesians': 79, 'Job': 267, 'Hebrews': 202, 'Matthew': 281, 'Jeremiah': 237, 'Revelation': 122, 'Exodus': 365, 'Nehemiah': 60, '1 Corinthians': 102, 'Proverbs': 330, 'Romans': 153, 'Zechariah': 34, '2 Peter': 31, 'Acts': 242, 'Ecclesiastes': 99, 'Mark': 44, '1 Chronicles': 139, 'Nahum': 6, 'John': 143, '2 Corinthians': 69, '1 Timothy': 68, 'Genesis': 3372, '1 Thessalonians': 35, 'Jonah': 20, 'Deuteronomy': 322, 'James': 41, 'Luke': 233, 'Galatians': 70, 'Amos': 37, 'Ezekiel': 166, 'Joel': 14, 'Habakkuk': 15, '1 Kings': 128, 'Leviticus': 105, 'Colossians': 36, 'Malachi': 32, 'Hosea': 58, 'Lamentations': 20, '2 Chronicles': 77, '2 Kings': 100, 'Numbers': 221, '1 Samuel': 256, 'Daniel': 99, 'Ruth': 48, '1 Peter': 69, 'Judges': 165, '2 Samuel': 191, 'Joshua': 145, '2 Thessalonians': 9, 'Micah': 43, 'Esther': 37, 'Titus': 25, 'Jude': 12, 'Zephaniah': 13, 'Song of Solomon': 27, 'Ezra': 19, '2 Timothy': 19, 'Philippians': 25, 'Haggai':

In [675]:
# print(json.dumps(dict_cr, indent=4))

In [676]:
# with open("../data/dict_cr.json", "w") as json_file:
#     json.dump(dict_cr, json_file, indent=4)

In [9]:
df_kjv = pd.read_csv("../data/kjv.csv")
df_kjv = df_kjv.astype({col: 'string' for col in df_kjv.select_dtypes(include='object').columns})

In [678]:
# print(df_kjv)

In [10]:
dict_kjv = defaultdict(lambda: defaultdict(list))

for _, row in df_kjv.iterrows():
    dict_kjv[row["book_name"]][row["chapter_number"]].append(row["verse_number"])

dict_kjv = {book: dict(chapters) for book, chapters in dict_kjv.items()}

In [680]:
# print(dict_kjv)

In [681]:
# books_kjv = [
#         'Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy', 'Joshua', 
#         'Judges', 'Ruth', '1 Samuel', '2 Samuel', '1 Kings', '2 Kings', 
#         '1 Chronicles', '2 Chronicles', 'Ezra', 'Nehemiah', 'Esther', 'Job', 
#         'Psalms', 'Proverbs', 'Ecclesiastes', 'Song of Solomon', 'Isaiah', 
#         'Jeremiah', 'Lamentations', 'Ezekiel', 'Daniel', 'Hosea', 'Joel', 'Amos', 
#         'Obadiah', 'Jonah', 'Micah', 'Nahum', 'Habakkuk', 'Zephaniah', 'Haggai', 
#         'Zechariah', 'Malachi', 'Matthew', 'Mark', 'Luke', 'John', 'Acts', 
#         'Romans', '1 Corinthians', '2 Corinthians', 'Galatians', 'Ephesians', 
#         'Philippians', 'Colossians', '1 Thessalonians', '2 Thessalonians', 
#         '1 Timothy', '2 Timothy', 'Titus', 'Philemon', 'Hebrews', 'James', 
#         '1 Peter', '2 Peter', '1 John', '2 John', '3 John', 'Jude', 'Revelation'
#     ]
    
books_kjv_abv = {
    "Genesis": "GEN",
    "Exodus": "EXO",
    "Leviticus": "LEV",
    "Numbers": "NUM",
    "Deuteronomy": "DEU",
    "Joshua": "JOS",
    "Judges": "JDG",
    "Ruth": "RUT",
    "1 Samuel": "1SA",
    "2 Samuel": "2SA",
    "1 Kings": "1KI",
    "2 Kings": "2KI",
    "1 Chronicles": "1CH",
    "2 Chronicles": "2CH",
    "Ezra": "EZR",
    "Nehemiah": "NEH",
    "Esther": "EST",
    "Job": "JOB",
    "Psalms": "PSA",
    "Proverbs": "PRO",
    "Ecclesiastes": "ECC",
    "Song of Solomon": "SNG",
    "Isaiah": "ISA",
    "Jeremiah": "JER",
    "Lamentations": "LAM",
    "Ezekiel": "EZK",
    "Daniel": "DAN",
    "Hosea": "HOS",
    "Joel": "JOL",
    "Amos": "AMO",
    "Obadiah": "OBA",
    "Jonah": "JON",
    "Micah": "MIC",
    "Nahum": "NAM",
    "Habakkuk": "HAB",
    "Zephaniah": "ZEP",
    "Haggai": "HAG",
    "Zechariah": "ZEC",
    "Malachi": "MAL",
    "Matthew": "MAT",
    "Mark": "MRK",
    "Luke": "LUK",
    "John": "JHN",
    "Acts": "ACT",
    "Romans": "ROM",
    "1 Corinthians": "1CO",
    "2 Corinthians": "2CO",
    "Galatians": "GAL",
    "Ephesians": "EPH",
    "Philippians": "PHP",
    "Colossians": "COL",
    "1 Thessalonians": "1TH",
    "2 Thessalonians": "2TH",
    "1 Timothy": "1TI",
    "2 Timothy": "2TI",
    "Titus": "TIT",
    "Philemon": "PHM",
    "Hebrews": "HEB",
    "James": "JAS",
    "1 Peter": "1PE",
    "2 Peter": "2PE",
    "1 John": "1JN",
    "2 John": "2JN",
    "3 John": "3JN",
    "Jude": "JUD",
    "Revelation": "REV"
}


countries_kjv = [
    'Syria', 'Jordan', 'Israel', 'Lebanon', 'Iraq', 'Egypt', 'Türkiye', 'Greece', 'North Macedonia', 
    'Italy', 'Libya', 'Cyprus', 'Saudi Arabia', 'Sudan', 'Djibouti', 'Iran', 'Yemen', 'Somalia', 'Azerbaijan', 
    'Armenia', 'Spain', 'Uganda', 'Tunisia', 'Croatia', 'Pakistan', 'Mozambique', 'India', 'Sri Lanka', 
    'Eritrea', 'Oman', 'Kuwait', 'Georgia', 'Bangladesh', 'Malta', 'United Arab Emirates'
]

In [682]:
old_testament = [
    "Genesis", "Exodus", "Leviticus", "Numbers", "Deuteronomy",
    "Joshua", "Judges", "Ruth", "1 Samuel", "2 Samuel",
    "1 Kings", "2 Kings", "1 Chronicles", "2 Chronicles", "Ezra",
    "Nehemiah", "Esther", "Job", "Psalms", "Proverbs",
    "Ecclesiastes", "Song of Solomon", "Isaiah", "Jeremiah", "Lamentations",
    "Ezekiel", "Daniel", "Hosea", "Joel", "Amos",
    "Obadiah", "Jonah", "Micah", "Nahum", "Habakkuk",
    "Zephaniah", "Haggai", "Zechariah", "Malachi"
]

new_testament = [
    "Matthew", "Mark", "Luke", "John", "Acts",
    "Romans", "1 Corinthians", "2 Corinthians", "Galatians", "Ephesians",
    "Philippians", "Colossians", "1 Thessalonians", "2 Thessalonians", "1 Timothy",
    "2 Timothy", "Titus", "Philemon", "Hebrews", "James",
    "1 Peter", "2 Peter", "1 John", "2 John", "3 John",
    "Jude", "Revelation"
]

books_kjv = old_testament + new_testament

In [683]:
# print(books_kjv)

In [684]:
dict_cr_ot_nt = {}

for book, references in dict_cr.items():
    if book in old_testament:
        filtered_references = {ref_book: count for ref_book, count in references.items() if ref_book in new_testament}
    elif book in new_testament:
        filtered_references = {ref_book: count for ref_book, count in references.items() if ref_book in old_testament}
    else:
        continue
    if filtered_references:
        dict_cr_ot_nt[book] = filtered_references

In [685]:
print(dict_cr_ot_nt)

{'Genesis': {'1 John': 33, 'Ephesians': 79, 'Hebrews': 202, 'Matthew': 281, 'Revelation': 122, '1 Corinthians': 102, 'Romans': 153, '2 Peter': 31, 'Acts': 242, 'Mark': 44, 'John': 143, '2 Corinthians': 69, '1 Timothy': 68, '1 Thessalonians': 35, 'James': 41, 'Luke': 233, 'Galatians': 70, 'Colossians': 36, '1 Peter': 69, '2 Thessalonians': 9, 'Titus': 25, 'Jude': 12, '2 Timothy': 19, 'Philippians': 25, 'Philemon': 3, '3 John': 1, '2 John': 1}, 'Exodus': {'Acts': 159, 'Titus': 12, 'James': 33, 'Romans': 79, 'John': 142, 'Revelation': 153, 'Matthew': 189, 'Luke': 129, 'Hebrews': 189, '1 Peter': 47, '1 John': 31, 'Galatians': 26, '1 Timothy': 27, 'Mark': 37, '2 Corinthians': 53, 'Ephesians': 57, 'Philippians': 21, '1 Corinthians': 106, '2 Thessalonians': 4, '2 Timothy': 18, 'Jude': 8, '2 Peter': 12, 'Colossians': 25, '1 Thessalonians': 17, '2 John': 1}, 'Leviticus': {'John': 84, 'Romans': 123, 'Ephesians': 55, '2 Corinthians': 76, 'Hebrews': 184, 'Luke': 87, '1 John': 42, 'Philippians': 18

In [686]:
dict_cr_unidirectional_ot = {}

for book1, references in dict_cr_ot_nt.items():
    if book1 in old_testament:
        for book2, count1 in references.items():
            dict_cr_unidirectional_ot[tuple(sorted((book1, book2)))] = count1
    else: pass

In [687]:
print(dict_cr_unidirectional_ot)

{('1 John', 'Genesis'): 33, ('Ephesians', 'Genesis'): 79, ('Genesis', 'Hebrews'): 202, ('Genesis', 'Matthew'): 281, ('Genesis', 'Revelation'): 122, ('1 Corinthians', 'Genesis'): 102, ('Genesis', 'Romans'): 153, ('2 Peter', 'Genesis'): 31, ('Acts', 'Genesis'): 242, ('Genesis', 'Mark'): 44, ('Genesis', 'John'): 143, ('2 Corinthians', 'Genesis'): 69, ('1 Timothy', 'Genesis'): 68, ('1 Thessalonians', 'Genesis'): 35, ('Genesis', 'James'): 41, ('Genesis', 'Luke'): 233, ('Galatians', 'Genesis'): 70, ('Colossians', 'Genesis'): 36, ('1 Peter', 'Genesis'): 69, ('2 Thessalonians', 'Genesis'): 9, ('Genesis', 'Titus'): 25, ('Genesis', 'Jude'): 12, ('2 Timothy', 'Genesis'): 19, ('Genesis', 'Philippians'): 25, ('Genesis', 'Philemon'): 3, ('3 John', 'Genesis'): 1, ('2 John', 'Genesis'): 1, ('Acts', 'Exodus'): 159, ('Exodus', 'Titus'): 12, ('Exodus', 'James'): 33, ('Exodus', 'Romans'): 79, ('Exodus', 'John'): 142, ('Exodus', 'Revelation'): 153, ('Exodus', 'Matthew'): 189, ('Exodus', 'Luke'): 129, ('Exo

In [688]:
dict_cr_unidirectional_ot_ord = {}

for (book1, book2), count in dict_cr_unidirectional_ot.items():
    if book1 in new_testament and book2 in old_testament:
        new_key = (book2, book1)
    else:
        new_key = (book1, book2)

    dict_cr_unidirectional_ot_ord[new_key] = count

In [689]:
print(dict_cr_unidirectional_ot_ord)

{('Genesis', '1 John'): 33, ('Genesis', 'Ephesians'): 79, ('Genesis', 'Hebrews'): 202, ('Genesis', 'Matthew'): 281, ('Genesis', 'Revelation'): 122, ('Genesis', '1 Corinthians'): 102, ('Genesis', 'Romans'): 153, ('Genesis', '2 Peter'): 31, ('Genesis', 'Acts'): 242, ('Genesis', 'Mark'): 44, ('Genesis', 'John'): 143, ('Genesis', '2 Corinthians'): 69, ('Genesis', '1 Timothy'): 68, ('Genesis', '1 Thessalonians'): 35, ('Genesis', 'James'): 41, ('Genesis', 'Luke'): 233, ('Genesis', 'Galatians'): 70, ('Genesis', 'Colossians'): 36, ('Genesis', '1 Peter'): 69, ('Genesis', '2 Thessalonians'): 9, ('Genesis', 'Titus'): 25, ('Genesis', 'Jude'): 12, ('Genesis', '2 Timothy'): 19, ('Genesis', 'Philippians'): 25, ('Genesis', 'Philemon'): 3, ('Genesis', '3 John'): 1, ('Genesis', '2 John'): 1, ('Exodus', 'Acts'): 159, ('Exodus', 'Titus'): 12, ('Exodus', 'James'): 33, ('Exodus', 'Romans'): 79, ('Exodus', 'John'): 142, ('Exodus', 'Revelation'): 153, ('Exodus', 'Matthew'): 189, ('Exodus', 'Luke'): 129, ('Exo

In [690]:
dict_cr_unidirectional_nt = {}

for book1, references in dict_cr_ot_nt.items():
    if book1 in new_testament:
        for book2, count1 in references.items():
            dict_cr_unidirectional_nt[tuple(sorted((book1, book2)))] = count1
    else: pass

In [691]:
print(dict_cr_unidirectional_nt)

{('Genesis', 'Matthew'): 140, ('Isaiah', 'Matthew'): 392, ('Matthew', 'Psalms'): 447, ('Jeremiah', 'Matthew'): 169, ('Amos', 'Matthew'): 15, ('Matthew', 'Zechariah'): 73, ('1 Chronicles', 'Matthew'): 27, ('Matthew', 'Numbers'): 47, ('Matthew', 'Ruth'): 12, ('1 Samuel', 'Matthew'): 71, ('2 Samuel', 'Matthew'): 43, ('1 Kings', 'Matthew'): 99, ('2 Chronicles', 'Matthew'): 44, ('2 Kings', 'Matthew'): 52, ('Daniel', 'Matthew'): 84, ('Ezra', 'Matthew'): 20, ('Haggai', 'Matthew'): 8, ('Matthew', 'Nehemiah'): 26, ('Job', 'Matthew'): 85, ('Leviticus', 'Matthew'): 55, ('Matthew', 'Proverbs'): 163, ('Joel', 'Matthew'): 10, ('Judges', 'Matthew'): 29, ('Exodus', 'Matthew'): 82, ('Matthew', 'Micah'): 27, ('Malachi', 'Matthew'): 47, ('Joshua', 'Matthew'): 24, ('Lamentations', 'Matthew'): 20, ('Deuteronomy', 'Matthew'): 86, ('Hosea', 'Matthew'): 42, ('Ezekiel', 'Matthew'): 95, ('Esther', 'Matthew'): 12, ('Matthew', 'Zephaniah'): 10, ('Matthew', 'Song of Solomon'): 16, ('Matthew', 'Obadiah'): 5, ('Matt

In [692]:
dict_cr_unidirectional_nt_ord = {}

for (book1, book2), count in dict_cr_unidirectional_nt.items():
    if book1 in old_testament and book2 in new_testament:
        new_key = (book2, book1)
    else:
        new_key = (book1, book2)

    dict_cr_unidirectional_nt_ord[new_key] = count

In [693]:
print(dict_cr_unidirectional_nt_ord)

{('Matthew', 'Genesis'): 140, ('Matthew', 'Isaiah'): 392, ('Matthew', 'Psalms'): 447, ('Matthew', 'Jeremiah'): 169, ('Matthew', 'Amos'): 15, ('Matthew', 'Zechariah'): 73, ('Matthew', '1 Chronicles'): 27, ('Matthew', 'Numbers'): 47, ('Matthew', 'Ruth'): 12, ('Matthew', '1 Samuel'): 71, ('Matthew', '2 Samuel'): 43, ('Matthew', '1 Kings'): 99, ('Matthew', '2 Chronicles'): 44, ('Matthew', '2 Kings'): 52, ('Matthew', 'Daniel'): 84, ('Matthew', 'Ezra'): 20, ('Matthew', 'Haggai'): 8, ('Matthew', 'Nehemiah'): 26, ('Matthew', 'Job'): 85, ('Matthew', 'Leviticus'): 55, ('Matthew', 'Proverbs'): 163, ('Matthew', 'Joel'): 10, ('Matthew', 'Judges'): 29, ('Matthew', 'Exodus'): 82, ('Matthew', 'Micah'): 27, ('Matthew', 'Malachi'): 47, ('Matthew', 'Joshua'): 24, ('Matthew', 'Lamentations'): 20, ('Matthew', 'Deuteronomy'): 86, ('Matthew', 'Hosea'): 42, ('Matthew', 'Ezekiel'): 95, ('Matthew', 'Esther'): 12, ('Matthew', 'Zephaniah'): 10, ('Matthew', 'Song of Solomon'): 16, ('Matthew', 'Obadiah'): 5, ('Matt

### Network Graph

In [694]:
import numpy as np
import pandas as pd

import networkx as nx
import plotly.graph_objects as go
from pyvis.network import Network

#### PyVis

In [695]:
G = nx.Graph()

for book in old_testament:
    G.add_node(book, group="Old Testament")
for book in new_testament:
    G.add_node(book, group="New Testament")

In [696]:
edge_weights = {}
for book, references in dict_cr.items():
    for ref_book, weight in references.items():
        edge = tuple(sorted((book, ref_book)))
        edge_weights[edge] = edge_weights.get(edge, 0) + weight

In [697]:
for edge, total_weight in edge_weights.items():
    G.add_edge(*edge, weight=total_weight)

In [698]:
out_ref_cr = {book: sum(references.values()) for book, references in dict_cr.items()}

in_ref_cr = {book: 0 for book in books_kjv}

for book, references in dict_cr.items():
    for ref_book, count in references.items():
        in_ref_cr[ref_book] = in_ref_cr.get(ref_book, 0) + count

In [699]:
net = Network(height="1500px", width="1500px", bgcolor="#0f1116", font_color="white", notebook=False)
net.from_nx(G)

net.toggle_physics(True)
net.show_buttons(filter_=['physics'])

net.barnes_hut()
# net.force_atlas_2based()

In [700]:
for node in net.nodes:
    book = node["id"]
    group = node.get("group", "")
    
    node["physics"] = True
    
    # node["borderWidth"] = 10
    
    node["label"] = book
    node["labelHighlightBold"] = True
    node["shape"] = "dot"
    node["size"] = 100
    
    node["font"]["size"] = 75
    
    if group == "Old Testament":
        node["color"] = "#e3191f"
    elif group == "New Testament":
        node["color"] = "#1e7ad1"
    else:
        node["color"] = "gray"
    
    out_refs = out_ref_cr.get(book, 0)
    in_refs = in_ref_cr.get(book, 0)
    node["title"] = (
        f"{book}\n"
        f"Incoming References: {in_refs}\n"
        f"Outgoing References: {out_refs}"
    )

In [701]:
for edge in net.edges:
    from_book = edge["from"]
    to_book = edge["to"]
    
    edge["title"] = f"1st Book: {from_book}\n2nd Book: {to_book}\nReferences: {edge['width']}"
    edge["physics"] = True
    
    width = edge.get("width", 1)
    edge["width"] = min(width / 50, 2.5)
    
    from_book = edge.get("from")
    to_book = edge.get("to")
    
    if from_book in old_testament and to_book in old_testament:
        edge["color"] = "#fc896a"
    elif from_book in new_testament and to_book in new_testament:
        edge["color"] = "#93c3df"
    else:
        edge["color"] = "#dbb8ff"

In [702]:
# net.save_graph("../models/network_cr_pv.html")

#### Plotly

In [703]:
G = nx.Graph()

for book in old_testament:
    G.add_node(book, group="Old Testament")
for book in new_testament:
    G.add_node(book, group="New Testament")

In [704]:
edge_weights = {}

for book, references in dict_cr.items():
    for ref_book, weight in references.items():
        edge = tuple(sorted((book, ref_book)))
        edge_weights[edge] = edge_weights.get(edge, 0) + weight

In [705]:
for edge, total_weight in edge_weights.items():
    G.add_edge(*edge, weight=total_weight)

In [706]:
# pos = nx.spring_layout(G, 
#                     seed=1, 
#                     k=5, 
#                     scale=None,
#                     iterations = 1000,
#                     dim=2)

# shift = 0.5
# for node in G.nodes():
#     if G.nodes[node]["group"] == 'Old Testament':
#         pos[node][0] -= shift
#         pos[node][1] -= shift
#     else:
#         pos[node][0] += shift
#         pos[node][1] += shift
        
        
# pos = nx.random_layout(G, seed=42)
# pos = nx.circular_layout(G, scale=2, center=(0, 0))
# pos = nx.kamada_kawai_layout(G, scale=2, center=(0, 0))
pos = nx.spiral_layout(G, scale=2, center=(0, 0), resolution=0.35, 
                    # equidistant=True
                    )

In [707]:
out_ref_cr = {book: sum(references.values()) for book, references in dict_cr.items()}

in_ref_cr = {}
for book, references in dict_cr.items():
    for referenced_book, count in references.items():
        if referenced_book not in in_ref_cr:
            in_ref_cr[referenced_book] = 0
        in_ref_cr[referenced_book] += count

In [708]:
node_connections = {book: out_ref + in_ref_cr[book] for book, out_ref in out_ref_cr.items()}

max_connections = max(node_connections.values()) if node_connections else 1

In [709]:
ot_nodes = [node for node in G.nodes() if G.nodes[node]['group'] == "Old Testament"]
nt_nodes = [node for node in G.nodes() if G.nodes[node]['group'] == "New Testament"]

In [710]:
ot_colors = [node_connections[node] for node in ot_nodes]
nt_colors = [node_connections[node] for node in nt_nodes]

In [711]:
ot_x = [pos[node][0] for node in ot_nodes]
ot_y = [pos[node][1] for node in ot_nodes]

ot_text = [f"{node} (Incoming References: {in_ref_cr[node]} | Outgoing References: {out_ref_cr[node]})" for node in ot_nodes]

In [712]:
nt_x = [pos[node][0] for node in nt_nodes]
nt_y = [pos[node][1] for node in nt_nodes]

nt_text = [f"{node} (Incoming References: {in_ref_cr[node]} | Outgoing References: {out_ref_cr[node]})" for node in nt_nodes]

In [713]:
ot_trace = go.Scatter(
    x=ot_x, y=ot_y,
    mode='markers+text',
    text=ot_nodes,
    textposition="top center",
    hoverinfo='text',
    hovertext=ot_text,
    marker=dict(
        size=15,
        color=ot_colors,
        colorscale="Reds",
        showscale=False,
        cmin=0,
        cmax=20000,
        line=dict(width=1, color="black")
    ),
    textfont=dict(color="white", size=12),
    name="Old Testament"
)

In [714]:
nt_trace = go.Scatter(
    x=nt_x, y=nt_y,
    mode='markers+text',
    text=nt_nodes,
    textposition="top center",
    hoverinfo='text',
    hovertext=nt_text,
    marker=dict(
        size=15,
        color=nt_colors,
        colorscale="Blues",
        showscale=False,
        cmin=0,
        cmax=20000,
        line=dict(width=1, color="black")
    ),
    textfont=dict(color="white", size=12),
    name="New Testament"
)

In [715]:
edge_traces = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    # x0, y0, z0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    # x1, y1, z1 = pos[edge[1]]
    weight = G.edges[edge]['weight']
    # width = weight / 50
    # if width > 2.5:
    #     width = 2.5
    width = 1
    
    from_book = edge[0]
    to_book = edge[1]
    
    if from_book in old_testament and to_book in old_testament:
        color = "#fc896a"
    elif from_book in new_testament and to_book in new_testament:
        color = "#93c3df"
    else:
        color = "#dbb8ff"
    
    edge_trace = go.Scatter(
        x=[x0, x1, None],
        # x=[x0, x1, z0],
        y=[y0, y1, None],
        # y=[y0, y1, z1],
        line=dict(width=width, color=color),
        hoverinfo='none',
        mode='lines'
    )
    edge_traces.append(edge_trace)

In [ ]:
fig = go.Figure(data=edge_traces + [ot_trace, nt_trace],
                layout=go.Layout(
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0, l=0, r=0, t=0),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    width=1000, height=1000,
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)'
                ))

In [ ]:
# fig.show()

In [718]:
# fig.write_json("../models/figure_cr_ng.json")
# fig.write_json("../models/figure_cr_ng_spiral.json")

### Chord Diagram

In [719]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go

import seaborn as sns
import matplotlib.colors as mcolors

In [720]:
book_index = {book: i for i, book in enumerate(books_kjv)}

In [721]:
colors_rgb = sns.color_palette("pastel", 66)
colors = [mcolors.to_hex(color) for color in colors_rgb]

In [722]:
n_books = len(books_kjv)
angles = np.linspace(0, 2 * np.pi, n_books, endpoint=False)
positions = {book: (np.cos(angle), np.sin(angle)) for book, angle in zip(books_kjv, angles)}

In [723]:
edges = []
for book, refs in dict_cr.items():
    if book not in book_index:
        continue
    book_pos = positions[book]
    
    for ref, weight in refs.items():
        if ref not in book_index:
            continue
        ref_pos = positions[ref]
        edge_color = colors[book_index[book] % len(colors)]
        
        curve = np.linspace(0, 1, 20)
        bezier_x = (1 - curve)**2 * book_pos[0] + 2 * (1 - curve) * curve * 0 + curve**2 * ref_pos[0]
        bezier_y = (1 - curve)**2 * book_pos[1] + 2 * (1 - curve) * curve * 0 + curve**2 * ref_pos[1]
        
        edges.append(go.Scatter(
            x=bezier_x, y=bezier_y, mode="lines",
            line=dict(color=edge_color, width=max(weight / 100, 0.5)),
            hoverinfo="none"
        ))

In [724]:
incoming_refs = {book: 0 for book in books_kjv}
outgoing_refs = {book: 0 for book in books_kjv}

for book, refs in dict_cr.items():
    if book in outgoing_refs:
        outgoing_refs[book] += sum(refs.values())
    for ref, count in refs.items():
        if ref in incoming_refs:
            incoming_refs[ref] += count

In [725]:
nodes = go.Scatter(
    x=[positions[book][0] for book in books_kjv],
    y=[positions[book][1] for book in books_kjv],
    mode="markers+text",
    text=[book for book in books_kjv],
    textfont=dict(size=10, color="white"),
    textposition="top center",
    hovertext=[f"{book}<br>Incoming References: {incoming_refs[book]} | Outgoing References: {outgoing_refs[book]}" for book in books_kjv],
    marker=dict(size=10, color="white"),
    hoverinfo="text"
)

In [726]:
layout = go.Layout(
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    width=1250, height=1250,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

In [ ]:
fig = go.Figure(data=edges + [nodes], layout=layout)

In [ ]:
# fig.show()

In [729]:
# fig.write_json("../models/figure_cr_cd.json")

### Sankey Diagram

In [730]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go

import seaborn as sns
import matplotlib.colors as mcolors

#### Left Nodes

In [731]:
dict_count_ot = {}

for (ot_book, nt_book), count in dict_cr_unidirectional_ot_ord.items():
    if ot_book in dict_count_ot:
        dict_count_ot[ot_book] += count
    else:
        dict_count_ot[ot_book] = count

In [732]:
dict_count_ot_ordered = {book: dict_count_ot[book] for book in old_testament if book in dict_count_ot}
dict_count_ot = dict_count_ot_ordered

In [733]:
print(dict_count_ot)

{'Genesis': 2148, 'Exodus': 1575, 'Leviticus': 1283, 'Numbers': 1070, 'Deuteronomy': 1313, 'Joshua': 474, 'Judges': 480, 'Ruth': 131, '1 Samuel': 743, '2 Samuel': 524, '1 Kings': 798, '2 Kings': 659, '1 Chronicles': 424, '2 Chronicles': 703, 'Ezra': 173, 'Nehemiah': 369, 'Esther': 146, 'Job': 1303, 'Psalms': 4810, 'Proverbs': 1699, 'Ecclesiastes': 431, 'Song of Solomon': 390, 'Isaiah': 3147, 'Jeremiah': 1687, 'Lamentations': 120, 'Ezekiel': 1429, 'Daniel': 963, 'Hosea': 348, 'Joel': 122, 'Amos': 177, 'Obadiah': 23, 'Jonah': 85, 'Micah': 225, 'Nahum': 49, 'Habakkuk': 94, 'Zephaniah': 116, 'Haggai': 63, 'Zechariah': 578, 'Malachi': 244}


In [734]:
dict_count_ot_ls = [0.5]

keys = list(dict_count_ot.keys())[:-1]

for key in keys:
    dict_count_ot_ls.append(dict_count_ot_ls[-1] + 100 + dict_count_ot[key]/2.5)

#### Right Nodes

In [735]:
dict_count_nt = {}

for (ot_book, nt_book), count in dict_cr_unidirectional_ot_ord.items():
    if nt_book in dict_count_nt:
        dict_count_nt[nt_book] += count
    else:
        dict_count_nt[nt_book] = count

In [736]:
dict_count_nt_ordered = {book: dict_count_nt[book] for book in new_testament if book in dict_count_nt}
dict_count_nt = dict_count_nt_ordered

In [737]:
print(dict_count_nt)

{'Matthew': 4198, 'Mark': 654, 'Luke': 2982, 'John': 2475, 'Acts': 2976, 'Romans': 2230, '1 Corinthians': 1335, '2 Corinthians': 1095, 'Galatians': 514, 'Ephesians': 1041, 'Philippians': 510, 'Colossians': 391, '1 Thessalonians': 416, '2 Thessalonians': 208, '1 Timothy': 551, '2 Timothy': 399, 'Titus': 264, 'Philemon': 18, 'Hebrews': 2238, 'James': 881, '1 Peter': 1073, '2 Peter': 420, '1 John': 503, '2 John': 27, '3 John': 20, 'Jude': 232, 'Revelation': 3465}


In [738]:
dict_count_nt_ls = [0.5]

keys = list(dict_count_nt.keys())[:-1]

for key in keys:
    dict_count_nt_ls.append(dict_count_nt_ls[-1] + 50 + dict_count_nt[key]/2)

#### Chart

In [739]:
book_indices = {book: i for i, book in enumerate(old_testament + new_testament)}

sources = [book_indices[ot] for ot, nt in dict_cr_unidirectional_ot_ord.keys()]
targets = [book_indices[nt] for ot, nt in dict_cr_unidirectional_ot_ord.keys()]
values = list(dict_cr_unidirectional_ot_ord.values())

In [740]:
node_customdata = []
for book in (old_testament + new_testament):
    if book in old_testament:
        outgoing_references = sum(values[i] for i in range(len(sources)) if sources[i] == book_indices[book])
        outgoing_edges = sum(1 for i in range(len(sources)) if sources[i] == book_indices[book])
        node_customdata.append(f"Outgoing References: {outgoing_references}<br>Outgoing Edges (# of New Testament Books): {outgoing_edges}")
    else:
        incoming_references = sum(values[i] for i in range(len(targets)) if targets[i] == book_indices[book])
        incoming_edges = sum(1 for i in range(len(targets)) if targets[i] == book_indices[book])
        node_customdata.append(f"Incoming References: {incoming_references}<br>Incoming Edges (# of Old Testament Books): {incoming_edges}")

In [741]:
link_customdata = [
    f"Book (Source): {list(book_indices.keys())[sources[i]]}<br>Book (Target):{list(book_indices.keys())[targets[i]]}<br># of References: {values[i]}"
    for i in range(len(sources))
]

In [742]:
x_positions = [0.001 for left in old_testament] + [0.999 for right in new_testament]

def min_max_normalize(lst):
    min_val = min(lst)
    max_val = max(lst)
    return [(x - min_val) / (max_val - min_val) for x in lst] if max_val != min_val else [0] * len(lst)

y_positions = min_max_normalize(dict_count_ot_ls) + min_max_normalize(dict_count_nt_ls)

y_positions[0] = 0.001
y_positions[39] = 0.025
y_positions[18] += + 0.0375

for y in range(len(y_positions)):
    if y > 39:
        # print(y_positions[y])
        y_positions[y] -= 0.0375
        # print(y_positions[y])

y_positions[40] -= 0.02
y_positions[42] -= 0.025

In [743]:
colors_rgb = sns.color_palette("pastel", 66)
colors = [mcolors.to_hex(color) for color in colors_rgb]

In [744]:
link_colors = [colors[sources[i]] for i in range(len(sources))]

In [ ]:
fig = go.Figure(
    data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=books_kjv,
            x=x_positions,
            y=y_positions,
            color=colors,
            customdata=node_customdata,
            hovertemplate='%{label}<br>%{customdata}<extra></extra>',
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values,
            color=link_colors,
            customdata=link_customdata,
            hovertemplate='%{customdata}<extra></extra>',
        )
    )],
    layout=dict(
        font_size=12,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=100, r=100, t=50, b=50),
        height=2000,
        width=1250,
    )
)

In [ ]:
# fig.show()

In [747]:
# fig.write_json("../models/figure_cr_sd.json")

### Heat Map

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [749]:
book_index = {book: i for i, book in enumerate(books_kjv)}
n_books = len(books_kjv)

In [750]:
heat_data = np.zeros((n_books, n_books))

In [751]:
for source_book, refs in dict_cr.items():
    if source_book not in book_index:
        continue
    i = book_index[source_book]
    for target_book, count in refs.items():
        if target_book not in book_index:
            continue
        j = book_index[target_book]
        heat_data[i, j] = count
        
heat_data = heat_data.T                

In [752]:
heatmap = go.Heatmap(
    z=heat_data,
    x=books_kjv,
    y=books_kjv,
    colorscale='Reds',
    hoverongaps=False,
    zmin=0,
    zmax=500,
    hovertemplate="Book (Source): %{x}<br>Book (Target): %{y}<br># of References: %{z}<extra></extra>"
)

In [753]:
layout = go.Layout(
    xaxis=dict(title="Book (Source)", tickangle=45),
    yaxis=dict(title="Book (Target)"),
    width=1250,
    height=1250,
)

In [ ]:
fig = go.Figure(data=[heatmap], layout=layout)

In [ ]:
# fig.show()

In [756]:
# fig.write_json("../models/figure_cr_hm.json")